In [64]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import math

%matplotlib inline

In [65]:
def time_to_seconds(time: str) -> int:
    h, m, s = map(int, time.split(':'))
    return s + m*60 + h*3600

In [66]:
def calc_angle(x1, y1, x2, y2, x3, y3):
    v1 = ((x2-x1), (y2-y1))
    v2 = ((x3-x2), (y3-y2))
    sqrt1, sqrt2 = np.sqrt([v1[0]**2+v1[1]**2, v2[0]**2+v2[1]**2])
    cos = (v1[0]*v2[0] + v1[1]*v2[1]) / (sqrt1*sqrt2)
    return (cos + 1) / 2

In [67]:
def calc_distance(x1, y1, x2, y2):
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

In [82]:
path = 'data\BadTracksHackaton1801.txt'

In [83]:
data = pd.read_csv(path, sep=' ', header=None, names=['time', 'id', 'latitude', 'longitude', 'elevation', 'code', 'name'])

In [84]:
data.head()

,time,id,latitude,longitude,elevation,code,name
0,0:00:01,8,49.722522,40.308650,11567,1025.0,TUM9102
1,0:00:11,8,49.737334,40.285857,11567,1025.0,TUM9102
2,0:00:21,8,49.752375,40.264201,11567,1025.0,TUM9102
3,0:00:31,8,49.767809,40.243869,11567,1025.0,TUM9102
4,0:00:41,8,49.785372,40.222412,11575,1025.0,TUM9102


In [85]:
data['time'] = data['time'].apply(time_to_seconds)

In [86]:
grouped = data.groupby('id')
grouped_features = []
for track_id, track_data in grouped:
    p_index = 0
    min_angle = 2
    max_z_speed = 0
    avg_z_speed = 0
    max_xy_speed = 0
    avg_xy_speed = 0
    max_time_diff = 0
    avg_time_diff = (track_data.iloc[track_data.shape[0] - 1]['time'] - track_data.iloc[0]['time']) / track_data.shape[0]
    for i in range(1, track_data.shape[0]):
        time_diff = track_data.iloc[i]['time'] - track_data.iloc[i - 1]['time']
        if(time_diff == 0):
            time_diff = .001
        if time_diff > max_time_diff:
            max_time_diff = time_diff
        
        if i != track_data.shape[0] - 1:
            angle = calc_angle(track_data.iloc[i - 1]['latitude'], track_data.iloc[i - 1]['longitude'], track_data.iloc[i]['latitude'],
                               track_data.iloc[i]['longitude'], track_data.iloc[i + 1]['latitude'], track_data.iloc[i + 1]['longitude'])
            if angle < min_angle:
                min_angle = angle
                p_index = i
        
        xy_speed = calc_distance(track_data.iloc[i - 1]['latitude'], track_data.iloc[i - 1]['longitude'], 
                                 track_data.iloc[i]['latitude'], track_data.iloc[i]['longitude']) / time_diff
        if xy_speed > max_xy_speed:
            max_xy_speed = xy_speed
        avg_xy_speed += xy_speed
        
        z_speed = abs(track_data.iloc[i-1]['elevation'] - track_data.iloc[i]['elevation']) / time_diff
        if z_speed > max_z_speed:
            max_z_speed = z_speed
        avg_z_speed += z_speed
        
    avg_xy_speed /= track_data.shape[0]
    avg_z_speed /= track_data.shape[0]
    grouped_features.append([min_angle, max_xy_speed, avg_xy_speed, max_z_speed, avg_z_speed, max_time_diff, avg_time_diff])
    break
grouped_features
    

[[0.0008693655189359606,
  2.099134345393234,
  0.007952598817526559,
  686.0,
  9.711568560241744,
  40,
  9.611731843575418]]